<b>Privacy Preserverd LBS Advertisement Using Private Proximity Testing With Homomorphic Encryption</b><br>
1.Alice wil send her encrypted zip and category to merchants.<br>
2.Merchants will encrypt and blind their zip code and category and encrypt the advertisement back to Alice.<br>
3.Alice will check if the merchant and Alice have same Zipcode and category.<br>
4.Neither Alice nor merchants knows the other parties zip or category and only knows if they are equal with her.<br>
5.Proximity is computed using XNOR function and will be 1 if they are equal and 0 if they are not .<br>
6.Alice multiply proximity (Either 0 or 1) with encrypted cipher text advertisement.<br>
7.Alice takes only merchants with same zip and category and discard rest of the merchants.<br>
8.Ciper text will be 0 for mismatched zip or  category afetr multiplication.<br>
9.Alice decrypt the advertisement and display in her UI or mobile application. <br><br>

<b>No Party knows the whereabouts of the other parties in the transaction. </b>

In [1]:
#Elgamal Encryption
import random
import hashlib
from Crypto.Util.number import inverse
import random
import sympy
from binascii import hexlify, unhexlify

# Convert string to integer representation
def string_to_int(string):
    return int.from_bytes(string.encode(), byteorder='big')
# Convert integer representation to string
def int_to_string(integer):
    return integer.to_bytes((integer.bit_length() + 7) // 8, byteorder='big').decode()
def xnor(a, b):
    return int(a == b)

# generate large prime number p
def generate_prime(bits):
    while True:
        p = random.getrandbits(bits)
        if is_prime(p):
            return p

# check if a number is prime
def is_prime(n):
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

# find a generator for the prime number p
def find_generator(p):
    for g in range(2, p):
        if pow(g, (p-1)//2, p) != 1 and pow(g, p-1, p) == 1:
            return g
    return None

def generate_keysSeperate():
    p1 =None
    while  p1 is None:
        p1 =sympy.randprime(random.getrandbits(500)*2,random.getrandbits(500)*4)# generate_prime(55)#(50)# generate_prime(1024)
    
    g1 =   find_generator(p1)
    x1 =  random.randint(1, p1-2)
    y1 =  pow(g1, x1, p1)
    return (p1, g1, y1), (p1, x1)

def generate_dynamicKeys(input_message):
    inputbytes = str.encode(input_message)
    m = int(hexlify(inputbytes), 16)
    p = sympy.randprime(m*2, m*4)
    g = sympy.randprime(int(m/2), m)
    x = random.randint(int(m/2),m)
    r = random.randint(int(m/2),m)
    y =  pow(g, x, p)
    return (p, g, y), (p, x)


# generate public and private keys
def generate_keys():
    p = generate_prime(50)# generate_prime(1024)
    g =   find_generator(p)
    x =  random.randint(1, p-2)
    y =  pow(g, x, p)
    return (p, g, y), (p, x)

# encrypt a message
def encrypt(public_key, message):
    p, g, y = public_key
    k = random.randint(1, p-2)
    r = pow(g, k, p)
    c = (message * pow(y, k, p)) % p
    return (r, c)

def decrypt (private_key, ciphertext):#(x,c1,c2,p):
    p, x = private_key
    c1, c2 = ciphertext
    s = pow(c1,x,p)
    dm = (c2 * inverse(s,p)) % p
    return dm


In [2]:
# generate public and private keys
Alice_public_key, Alice_private_key = generate_keysSeperate()


In [3]:
#Private Proximity detection using homomorphic encryption
def CalculateProximity(Alice_public_key,Alice_private_key,Alicevalue,BobValue):
    #Alice Send Value
    p, g, h =Alice_public_key  #(p, g, y), 
    p1, x= Alice_private_key   #(p, x)    
    r=random.randint(1, p-2)
    C1a= pow(g,r,p)
    C2a= pow(h, (Alicevalue+r),p) 
    
    #Bob Append And Send Value
    s=random.randint(1, p-2)
    t=random.randint(1, p-2)
    C1b= (pow(C1a, s  ,p) * pow(g,t,p)) %p 
    C2b= (pow(C2a, s  ,p) * pow(h,(t-s*BobValue),p)) %p
    
    #Alice Calculate Proximity
    p1, x= Alice_private_key 
    m=C2b/ pow(C1b,x,p)
    #m=m-1
    return m

In [4]:
#Merchants sending advertisement to Alice
from binascii import hexlify, unhexlify
AliceZipCode=78717
AliceCategory=4
lstAliceAdd=[]
lstAliceNotAdd=[]

Merchants = [(78717, 4, "Mcdonald's-World's best Burger"),
           (78718, 5, "Tacco Bel "),
           (78717, 4, "Wlmart -Price at lowest"),
            (78717, 6, "Sushi Bar -Best Sushi In Town!")]
for item in Merchants:
    MerchantZipCode, MerchantCategory, MerchantAd = item
    message_int = string_to_int(MerchantAd)
   
    #Calculate Zipcode proximity between Alice And Merchant
    ZipProximity=CalculateProximity(Alice_public_key,Alice_private_key,AliceZipCode,MerchantZipCode)
    
    #Calculate Category proximity between Alice And Merchant
    CategoryProximity=CalculateProximity(Alice_public_key,Alice_private_key,AliceCategory,MerchantCategory)

    #Encrypt the Merchant Advertisement
    EncryptedAd=encrypt(Alice_public_key, message_int)
    Rvalue,CipherValue=EncryptedAd

    #If Not same  Zip and Category betwen Alice and Merchant  discard proximity with XNOR function
    ZipProximityXNOR=xnor(ZipProximity,1)
    CategoryProximityXNOR=xnor(CategoryProximity,1)
    
    #Multiply proximity with Encrypted Cipher value. 
    #This will discard all encrypted Advertisement that are not proximate with Zip and Category
    CipherValue=int(CipherValue*ZipProximityXNOR*CategoryProximityXNOR)

    #decrypt advertisement that are only proximate with Zip and Category
    dm=decrypt(Alice_private_key, (Rvalue,CipherValue))
    x = format(dm, 'x')
    #If Not same Zip and Category Ciper Value will be 0 after multiplying with XNOR function
    if(CipherValue==0):
        lstAliceNotAdd.append(item)
        #print('You are not the recipient')
    else:
        decryptedAdd=unhexlify(x).decode("utf-8")
        lstAliceAdd.append(decryptedAdd)

print('Ad for Alice:')
for strAd in lstAliceAdd:
    print(strAd)
print('\nBelow are not selected and unknown to Alice:')
for strAd in lstAliceNotAdd:
    print(strAd)

Ad for Alice:
Mcdonald's-World's best Burger
Wlmart -Price at lowest

Below are not selected and unknown to Alice:
(78718, 5, 'Tacco Bel ')
(78717, 6, 'Sushi Bar -Best Sushi In Town!')
